# 핵심 키워드 추출 (Keyword Extraction)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 패키지 업그레이드로 인한 에러 처리를 위해 라이브러리 버전 Downgrade
!pip install 'networkx<2.7'

# Mecab 설치
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

# 런타임 재시작 !

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

# 1.1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tf_idf_vector.tocoo().data


---


## 실습 2. gensim 활용


### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(value) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tfidf[doc]



---



# 1.2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용


### 1) 토큰화 (Tokenization)

분석 텍스트 정제

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex)
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

### 5) 핵심 단어 추출

---

## 실습 2. 그래프 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(nodes, tokens) :
    window_size = 2 # coocurrence를 판단하기 위한 window 사이즈 설정

    edges = []
    for window_start in range(0,(len(tokens)-window_size+1)):
        window = tokens[window_start:window_start+window_size]
        for i in range(window_size) :
            for j in range(i+1, window_size) :
                if (window[i] in nodes) & (window[j] in nodes) : #윈도 내 단어가 노드에 속하는 경우만 엣지로 연결
                    edges.append((window[i], window[j]))
                    print((window[i], window[j]))
    return edges

# tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
# nodes = ['바나나', '사과', '파인애플', '딸기']

graph=nx.diamond_graph()
graph.clear()
graph.add_nodes_from(list(set(nodes))) #node 등록
graph.add_edges_from(connect(nodes, tokens)) #edge 연결

### 3) 스코어 계산 및 핵심키워드 추출

In [ ]:
scores = nx.pagerank(graph) #pagerank 계산
rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:5]

In [ ]:
import networkx as nx
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

d = to_pydot(graph)
d.set_dpi(600)
d.set_rankdir("LR")
Image(d.create_png(), width=600)

---

## 실습 3. TextRank 핵심 구 추출

### 1) 불용어를 기준으로 구 추출

In [ ]:
phrases = []
phrase = ' '

for word in tokens:
    if word[1][0] != 'N':
        if phrase != ' ':
            phrases.append(phrase.strip())
        phrase= ' '
    else:
        phrase+=word[0]
        phrase+= ' '

print(phrases)

In [ ]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print(unique_phrases)

### 2) 각 구의 Score 계산

앞서 산출한 각 단어별 점수를 합산

In [ ]:
vocabs = dict((r[0][0], r[1]) for r in rank)
phrase_scores = []
keywords = []

for phrase in unique_phrases:
    phrase_score = 0
    for word in phrase.split():
        if word in vocabs.keys():
            phrase_score+=vocabs[word]

    phrase_scores.append((phrase, phrase_score))

for phrase_score in phrase_scores[:10]:
    print(f"keyword: {phrase_score[0]}, Score : {phrase_score[1]}")

### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
sorted_phrase_score = sorted(phrase_scores, key=lambda tup: tup[1], reverse=True)
sorted_phrase_score[:10]



---



## 실습 4. gensim 활용

- summarization.keywords(document, words) : document를 요약하여 words만큼 키워드를 추출

In [ ]:
from gensim.summarization import keywords
keywords(doc, words=5).split('\n')